In [2]:
import operator
from numpy import load
import faiss 

import pandas as pd
import numpy as np
import os 
import re

import nltk 
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn

In [3]:
df_hot=pd.read_csv("hotels.csv")
df_resto=pd.read_csv('restaurants.csv')
df_att=pd.read_csv('attractions.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'hotels.csv'

In [ ]:
def search_bert(query,cat):
    
    if cat==1:  #Hotels
        query.replace("stars","")
        
        sbert_model = SentenceTransformer("sbert_model") 
        sentence_embeddings = load('bert_embeddings.npy')
        d=sentence_embeddings.shape[1]
        index = faiss.IndexFlatL2(d)   
        index.train(sentence_embeddings) 
        index.add(sentence_embeddings) 
        xb=sbert_model.encode([query])
        D,I=index.search(xb,10)
        sentences=df_hot['info'].to_list()
        return D,I,sentences
    elif cat==2: #restaurants
        
        sbert_model = SentenceTransformer("sbert_model") 
        sentence_embeddings = load('rbert_embeddings.npy')
        d=sentence_embeddings.shape[1]
        index = faiss.IndexFlatL2(d)   
        index.train(sentence_embeddings) 
        index.add(sentence_embeddings) 
        xb=sbert_model.encode([query])
        D,I=index.search(xb,10)
        sentences=df_att['info'].to_list()
        return D,I,sentences
    
    elif cat==3:
        
        sbert_model = SentenceTransformer("sbert_model") 
        sentence_embeddings = load('abert_embeddings.npy')
        d=sentence_embeddings.shape[1]
        index = faiss.IndexFlatL2(d)   
        index.train(sentence_embeddings) 
        index.add(sentence_embeddings) 
        xb=sbert_model.encode([query])
        D,I=index.search(xb,10)
        sentences=df_att['info'].to_list()
        return D,I,sentences

In [ ]:
%%time
D,I,sentences=search_bert(query,3)
# D,I,sentences=search_tfidf("jamaa elfna",3)
for i in range(10): 
            print("Sentence = ", sentences[I[0,i]])
            print("-----------------------------------------------------------\n") 

In [ ]:
def return_res(query):
    df_res=pd.DataFrame()
    D,I,sentences=search_bert(query,1)
    for i in range(10):
            df_res=df_res.append(df_hot[int(I[0,i]):int(I[0,i])+1], ignore_index=True) 
            print("-----------------------------------------------------------\n")
    data = df_res.to_json(orient='index')
    data=str(data)
    data=data.replace('"', '\\"')
    return data